In [ ]:
from keras.models import load_model
import numpy as np

# Giả sử bạn đã có mô hình FaceNet pretrained
model = load_model('model-cnn-facerecognition.h5')

# Lưu model
model.save('my_model.h5')

In [ ]:
from keras.models import load_model
import numpy as np
import cv2
from scipy.spatial.distance import euclidean

# Tải mô hình đã huấn luyện
model = load_model('my_model.h5')

# In cấu trúc mô hình để kiểm tra yêu cầu đầu vào
# model.summary()

# Chức năng để lấy embedding từ hình ảnh khuôn mặt
def get_embedding(model, face_pixels):
    # Đảm bảo kích thước ảnh là (160, 160, 3)
    required_size = (50, 50)
    face_pixels = cv2.resize(face_pixels, required_size)
    
    # Chuyển đổi ảnh màu thành ảnh xám
    face_gray = cv2.cvtColor(face_pixels, cv2.COLOR_BGR2GRAY)
    
    # Chuyển đổi sang float32 và chuẩn hóa
    face_gray = face_gray.astype('float32')
    mean, std = face_gray.mean(), face_gray.std()
    face_gray = (face_gray - mean) / std
    
    # Thêm chiều batch
    samples = np.expand_dims(face_gray, axis=0)

    try:
        yhat = model.predict(samples)

        # Đảm bảo đầu ra là một vector 1-D
        if len(yhat.shape) > 1:
            yhat = yhat.flatten()

    except Exception as e:
        print(f"Error during model prediction: {e}")
        yhat = None  # Gán giá trị None cho yhat nếu có lỗi
    
    return yhat if yhat is not None else np.array([])

# Hàm đăng ký người dùng
def register_user(user_id, face_pixels):
    # Chuyển đổi ảnh khuôn mặt thành embedding
    embedding = get_embedding(model, face_pixels)
    
    # Lưu trữ embedding vào cơ sở dữ liệu (giả sử sử dụng dictionary để lưu trữ)
    user_database[user_id] = embedding

# Hàm nhận dạng người dùng
def recognize_user(input_face_pixels):
    # Chuyển đổi ảnh khuôn mặt thành embedding
    input_embedding = get_embedding(model, input_face_pixels)
    print("Input embed: ", input_embedding)

    if input_embedding.size == 0:
        print("Error: Input embedding is empty.")
        return None

    min_dist = float('inf')
    identity = None
    for user_id, user_embedding in user_database.items():
        dist = euclidean(input_embedding, user_embedding)
        if dist < min_dist:
            min_dist = dist
            identity = user_id
    return identity

# Tạo cơ sở dữ liệu người dùng (giả sử đã có embedding của người dùng)
user_database = {}

# Đăng ký người dùng với một số ảnh khuôn mặt
register_user('user_1', cv2.imread('new_user_face.jpg'))
register_user('user_2', cv2.imread('new_user_face_2.jpg'))

# Nhận dạng người dùng từ ảnh nhận dạng
recognized_user = recognize_user(cv2.imread('input_face.jpg'))
print(f"Recognized user: {recognized_user}")